In [665]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd

pd.set_option('display.max_rows', 30000)

In [666]:
dffull=pd.read_csv('fulldf.csv')
dffull.head(1)

,Unnamed: 0,vertices,neg,ne,po,m,rumor,not_rumor,v1,v2,key1,key2,max,max_label,key1.1,key2.1
0,0,"('#rt', '#believe')",6,8,0,0,1,13,#rt,#believe,209,1326,8,1,209,1326


In [667]:
dffull.shape

(43004, 16)

In [668]:
#Add rumor probability column.
dffull['rumorprob']=dffull['rumor']/(dffull['rumor']+dffull['not_rumor'])
dffull.head(1)

,Unnamed: 0,vertices,neg,ne,po,m,rumor,not_rumor,v1,v2,key1,key2,max,max_label,key1.1,key2.1,rumorprob
0,0,"('#rt', '#believe')",6,8,0,0,1,13,#rt,#believe,209,1326,8,1,209,1326,0.071429


In [669]:
# Add positive probability column.
dffull['positive_prob']=dffull['po']/(dffull['po']++dffullnode['ne']+dffull['neg']+dffullnode['m'])

In [670]:
# define function to devide positive probability into two groups, using the mean positve probability of all nodes in
#entire dataset as threshold.
def node_sent_group(po_prob):
    if po_prob>NodePoMean:
        return 1  # high rumor prob
    else:
        return 0  # low rumor prob

In [671]:
# add sentiment group column for graphing later.
dffull['sent_group']=dffull['positive_prob'].apply(lambda x:node_sent_group(x))

In [672]:
# define function to devide positive probability into two groups, using the mean positve probability of all links in
#entire dataset as threshold.
def rumor_group(rumor_prob):
    if rumor_prob>LinkRumorMean:
        return 1  # high rumor prob
    else:
        return 0  # low rumor prob

In [673]:
# add rumor group column for graphing later.
dffull['rumor_group']=dffull['rumorprob'].apply(lambda x:node_rumor_group(x))

In [674]:
dffull.head(2)

,Unnamed: 0,vertices,neg,ne,po,m,rumor,not_rumor,v1,v2,key1,key2,max,max_label,key1.1,key2.1,rumorprob,positive_prob,sent_group,rumor_group
0,0,"('#rt', '#believe')",6,8,0,0,1,13,#rt,#believe,209,1326,8,1,209,1326,0.071429,0.000000,0,1
1,1,"('#rt', '#breathe')",1,2,1,0,0,4,#rt,#breathe,209,1475,2,1,209,1475,0.000000,0.047619,0,0


In [675]:
dffull.drop('Unnamed: 0',axis=1,inplace=True)

In [676]:
dffull.head(2)

,vertices,neg,ne,po,m,rumor,not_rumor,v1,v2,key1,key2,max,max_label,key1.1,key2.1,rumorprob,positive_prob,sent_group,rumor_group
0,"('#rt', '#believe')",6,8,0,0,1,13,#rt,#believe,209,1326,8,1,209,1326,0.071429,0.000000,0,1
1,"('#rt', '#breathe')",1,2,1,0,0,4,#rt,#breathe,209,1475,2,1,209,1475,0.000000,0.047619,0,0


In [677]:
LinkRumorMean=dffull['rumorprob'].mean()
LinkRumorMean

0.037905720720571266

In [678]:
dfhigh=pd.read_csv('Links_high.csv')
dfhigh.head(1)

,Unnamed: 0,vertices,neg,po,m,rumor,not_rumor,v1,v2,key1,key2,max,max_label
0,0,"('#rt', '#believe')",6,0,0,1,13,#rt,#believe,208,1325,8,1


In [679]:
dfhigh.drop(['key1','key2','Unnamed: 0'],axis=1,inplace=True)

In [680]:
dfhigh.shape

(2074, 10)

In [681]:
l=list(dfhigh.v1)+list(dfhigh.v2)
len(l)

4148

In [682]:
s=set(l)
l=list(s)
len(l)

409

In [683]:
dfnode=pd.DataFrame()

In [684]:
dfnode['tag']=l

In [685]:
dfnode['key']=[i for i in range(0,len(l))]
dfnode.shape

(409, 2)

In [686]:
dfnode.head(1)

,tag,key
0,#everything,0


In [687]:
dfhigh=dfhigh.merge(dfnode,how='left',left_on='v1',right_on='tag')

In [688]:
dfhigh.head(1)

,vertices,neg,po,m,rumor,not_rumor,v1,v2,max,max_label,tag,key
0,"('#rt', '#believe')",6,0,0,1,13,#rt,#believe,8,1,#rt,4


In [689]:
dfhigh.shape

(2074, 12)

In [690]:
dfhigh.rename(columns={'key':'key1'},inplace=True)

In [691]:
dfhigh=dfhigh.merge(dfnode,how='left',left_on='v2',right_on='tag')

In [692]:
dfhigh.rename(columns={'key':'key2'},inplace=True)
dfhigh.head(5)

,vertices,neg,po,m,rumor,not_rumor,v1,v2,max,max_label,tag_x,key1,tag_y,key2
0,"('#rt', '#believe')",6,0,0,1,13,#rt,#believe,8,1,#rt,4,#believe,307
1,"('#one', '#survival')",6,0,0,1,5,#one,#survival,6,0,#one,71,#survival,17
2,"('#rt', '#flu')",5,0,0,3,9,#rt,#flu,7,1,#rt,4,#flu,350
3,"('#rt', '#season')",5,0,0,1,7,#rt,#season,5,0,#rt,4,#season,166
4,"('#rt', '#corrupt')",1,0,0,1,0,#rt,#corrupt,1,0,#rt,4,#corrupt,89


In [693]:
dfhigh.drop(['tag_x','tag_y'],axis=1,inplace=True)

In [694]:
dfhigh['rumorprob']=dfhigh['rumor']/(dfhigh['rumor']+dfhigh['not_rumor'])

In [695]:
dfhigh['rumor_group']=dfhigh['rumorprob'].apply(lambda x: rumor_group(x))
dfhigh.reset_index(inplace=True)

In [696]:
#Set Colors for links. for forcenetworkD3 graphing later.
import pandas as pd
dfLinks_highrumor=pd.read_csv('dfhigh.csv')
def getcolor(rprob):
    if rprob>0.5:
        return 'hotpink'
    else:
        return'lightblue'
dfhigh['color']=dfhigh['rumorprob'].apply(lambda x:getcolor(x))

In [697]:
dfhigh.head(2)

,index,vertices,neg,po,m,rumor,not_rumor,v1,v2,max,max_label,key1,key2,rumorprob,rumor_group,color
0,0,"('#rt', '#believe')",6,0,0,1,13,#rt,#believe,8,1,4,307,0.071429,1,lightblue
1,1,"('#one', '#survival')",6,0,0,1,5,#one,#survival,6,0,71,17,0.166667,1,lightblue


In [698]:
dffullnode=pd.read_csv('nodeInfo2.csv')

In [699]:
dffullnode.rename(columns={'Unnamed: 0':'tag'},inplace=True)

In [700]:
dffullnode['rumorprob']=dffullnode['rumor']/(dffullnode['rumor']+dffullnode['not_rumor'])
dffullnode.head(1)

,tag,tag,tag.1,neg,ne,po,m,rumor,not_rumor,rumorprob,rumor_group
0,0,0,#rt,1634,1764,237,4,71,3568,0.019511,0


In [701]:
NodeRumorMean=dffullnode['rumorprob'].mean()
NodeRumorMean #Mean rumor prob of all nodes in dataset.

0.03691432435683804

In [702]:
dffullnode['positive_prob']=dffullnode['po']/(dffullnode['po']+dffullnode['neg']+dffullnode['m'])

In [703]:
NodePoMean=dffullnode['positive_prob'].mean()
NodePoMean

0.2284587620880507

In [704]:
dffullnode['sent_group']=dffullnode['positive_prob'].apply(lambda x:node_rumor_group(x))

In [705]:
def node_rumor_group(rumor_prob):
    if rumor_prob>NodeRumorMean:
        return 1  # high rumor prob
    else:
        return 0  # low rumor prob

In [706]:
dffullnode['rumor_group']=dffullnode['rumorprob'].apply(lambda x:node_rumor_group(x))

In [707]:
dffullnode.head(1)

,tag,tag,tag.1,neg,ne,po,m,rumor,not_rumor,rumorprob,rumor_group,positive_prob,sent_group
0,0,0,#rt,1634,1764,237,4,71,3568,0.019511,0,0.1264,1


In [708]:
dfnode =dfnode.merge(dffullnode, how='left',left_on='tag',right_on='tag.1')

In [709]:
dfnode.head(1)

,tag_x,key,tag_y,tag_y,tag.1,neg,ne,po,m,rumor,not_rumor,rumorprob,rumor_group,positive_prob,sent_group
0,#everything,0,569,569,#everything,13,3,2,1,2,17,0.105263,1,0.125,1


In [710]:
dfnode.drop(['tag_x','tag_y'],axis=1,inplace=True)

In [711]:
dfnode.rename(columns={'tag.1':'tag'},inplace=True)

In [712]:
dfnode.head(1)

,key,tag,neg,ne,po,m,rumor,not_rumor,rumorprob,rumor_group,positive_prob,sent_group
0,0,#everything,13,3,2,1,2,17,0.105263,1,0.125,1


In [713]:
dfhigh.to_csv('dfhigh.csv')

In [714]:
dfnode.to_csv('dfnode.csv')

In [715]:
dffullnode.to_csv('nodeInfo3.csv')

In [716]:
dff=pd.read_csv('nodeInfo3.csv')

In [717]:
dff.head()

,Unnamed: 0,tag,tag.1,tag.1.1,neg,ne,po,m,rumor,not_rumor,rumorprob,rumor_group,positive_prob,sent_group
0,0,0,0,#rt,1634,1764,237,4,71,3568,0.019511,0,0.1264,1
1,1,1,1,#believe,21,19,3,0,5,38,0.116279,1,0.1250,1
2,2,2,2,#breathe,2,3,2,0,0,7,0.000000,0,0.5000,1
3,3,3,3,#mind,1,4,0,0,0,5,0.000000,0,0.0000,0
4,4,4,4,#learning,3,14,1,0,1,17,0.055556,1,0.2500,1


In [718]:
dff=dff[['tag.1.1','neg','ne','po','m','rumor','not_rumor','rumorprob','rumor_group','positive_prob','sent_group']]

In [719]:
dff.rename(columns={'tag.1.1':'tag'},inplace=True)

In [720]:
dff.head(1)

,tag,neg,ne,po,m,rumor,not_rumor,rumorprob,rumor_group,positive_prob,sent_group
0,#rt,1634,1764,237,4,71,3568,0.019511,0,0.1264,1


In [721]:
dff.to_csv('nodeInfo_wgroups.csv')

In [722]:
dffull.to_csv('linkInfo_wgroups.csv')